In [7]:
import pandas as pd
import numpy as np
import random

# File operations
from copy import deepcopy
import glob
import os
from pathlib import Path

# Stats
from scipy.spatial.distance import cdist
from scipy.stats import spearmanr, wilcoxon, mannwhitneyu, zscore
import statsmodels.stats.multitest

# Neuromaps
import neuromaps
from neuromaps.parcellate import Parcellater
from neuromaps import datasets, images, nulls, resampling, stats
from nibabel import freesurfer as fs
import nibabel as nib
from neuromaps.datasets import fetch_annotation, fetch_fslr
from enigmatoolbox.datasets import load_sc, load_fc, load_fc_as_one
from nilearn.image import resample_to_img

# Set seed to 127
random.seed(127)

# Add connectome workbench to path
os.environ['PATH'] = os.environ['PATH'] + ':/Applications/workbench/bin_macosx64'

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
%%R 

suppressPackageStartupMessages({
    library(broom)
    library(circlize)
    library(ComplexHeatmap)
    library(cowplot)
    library(dendextend)
    library(FactoMineR)
    library(GGally)
    library(ggseg)
    library(glue)
    library(grid)
    library(patchwork)
    library(see)
    library(tidyverse)
})

# Set cowplot theme
theme_set(theme_cowplot())

In [67]:
data_path = "/Users/abry4213/github/Homotopic_FC/data/"

#### GROUP-AVERAGED FUNCTIONAL CONNECTOME FROM ENIGMA #### 
# Load cortico-cortical functional connectivity data
fc_ctx, fc_ctx_labels, _, _ = load_fc(parcellation='aparc')

# Read in preprocessed Pearson-based functional connectivity data from ENIGMA
HCP_ctx_avg_Pearson_FC = (pd.DataFrame(fc_ctx, columns=fc_ctx_labels)
                      .assign(Brain_Region_From = fc_ctx_labels)
                      .melt(id_vars=["Brain_Region_From"], 
                             var_name="Brain_Region_To", 
                             value_name="Mean_Pearson_R")
                      .assign(Base_Region_From = lambda x: x['Brain_Region_From'].str.replace("L_", "").str.replace("R_", ""),
                              Base_Region_To = lambda x: x['Brain_Region_To'].str.replace("L_", "").str.replace("R_", ""),
                              Hemi_From = lambda x: np.where(x['Brain_Region_From'].str.startswith("L_"), "Left", "Right"),
                              Hemi_To = lambda x: np.where(x['Brain_Region_To'].str.startswith("L_"), "Left", "Right"))
)

pearson_homotopic_res_mean = (HCP_ctx_avg_Pearson_FC
                              .query("Base_Region_From == Base_Region_To")
                              .groupby(["Base_Region_From"])['Mean_Pearson_R']
                              .mean() 
                              .reset_index()
                              .rename(columns={"Base_Region_From": "Base_Region"})
                              )

Read in neuroimaging data:

In [61]:
if not os.path.isfile(f"{data_path}/Arterial_Venous_Mean_Density_aparc.csv"):

    # aparc + aseg volume
    ROI_image = nib.load("/Users/abry4213/data/neuroimaging_atlases/volumes/freesurfer_segmentations/mni152_space/atlas-desikankilliany.nii.gz")
    ROI_lookup = pd.read_csv("/Users/abry4213/data/neuroimaging_atlases/volumes/freesurfer_segmentations/mni152_space/atlas-desikankilliany.csv")

    # Read in vasculature volumes (arterial and venous)
    arterial_image = nib.load(f"{data_path}/vasculature/mean_Ved_ToF_filt.nii.gz")
    venous_image = nib.load(f"{data_path}/vasculature/mean_Ved_swi_filt.nii.gz")

    # Downsample the segmentation volume to match the vasculature images
    ROI_image_resampled = resample_to_img(ROI_image, arterial_image, interpolation='nearest')
    ROI_data_resampled = ROI_image_resampled.get_fdata().flatten()

    # Extract data arrays
    venous_data = venous_image.get_fdata().flatten()
    arterial_data = arterial_image.get_fdata().flatten()

    ROI_indices = [int(i) for i in np.unique(ROI_data_resampled) if i != 0]

    regional_vascular_data_list = [] 

    for this_index in ROI_indices:
        region_info = ROI_lookup.query("id == @this_index")[['id', 'label', 'hemisphere']]

        arterial_roi_data = np.mean(arterial_data[ROI_data_resampled == this_index])
        venous_roi_data = np.mean(venous_data[ROI_data_resampled == this_index])

        ROI_vascular_df = pd.DataFrame({"id": this_index,
                                        "Arterial_Vascular": arterial_roi_data,
                                        "Venous_Vascular": venous_roi_data}, index=[0]).merge(region_info, on='id', how='left')
        
        regional_vascular_data_list.append(ROI_vascular_df)

    regional_vascular_data = (pd.concat(regional_vascular_data_list, ignore_index=True)
                            .rename(columns={"label": "Base_Region",
                                            'hemisphere': "Hemisphere"})
    )

    # Save to CSV
    regional_vascular_data.to_csv(f"{data_path}/Arterial_Venous_Mean_Density_aparc.csv", index=False)

else: 
    regional_vascular_data = pd.read_csv(f"{data_path}/Arterial_Venous_Mean_Density_aparc.csv")

# Take the mean of left/right
regional_vascular_data = regional_vascular_data.groupby(['Base_Region'])[['Arterial_Vascular', 'Venous_Vascular']].mean().reset_index()

In [83]:
%%R -i regional_vascular_data

### Pearson FC in brain ###
regional_vascular_data %>%
  ungroup() %>%
  mutate(label = glue("lh_{Base_Region}")) %>%
  left_join(., as_tibble(dk)) %>%
  ggseg(atlas = dk, mapping = aes(fill = Arterial_Vascular),
        position = "stacked", colour = "black", hemisphere="left",
        linewidth=0.5) +
  theme_void() +
  labs(fill = "Mean arterial density") +
  theme(plot.title = element_blank(),
        legend.position = "bottom") +
  scale_fill_gradient(low = "white", high = "red", na.value="white")

ggsave("arterial.png", width=5, height=3, units='in', dpi=300)

Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
In addition: Warning message:
Some data not merged properly. Check for naming errors in data:
  atlas type  hemi  side  region label       roi   Base_Region Arterial_Vascular
  <chr> <chr> <chr> <chr> <chr>  <glue>      <chr> <chr>                   <dbl>
1 <NA>  <NA>  <NA>  <NA>  <NA>   lh_accumbe… <NA>  accumbensa…              41.7
2 <NA>  <NA>  <NA>  <NA>  <NA>   lh_amygdala <NA>  amygdala                 22.4
3 <NA>  <NA>  <NA>  <NA>  <NA>   lh_brainst… <NA>  brainstem                46.9
4 <NA>  <NA>  <NA>  <NA>  <NA>   lh_caudate  <NA>  caudate                  14.9
5 <NA>  <NA>  <NA>  <NA>  <NA>   lh_hippoca… <NA>  hippocampus              21.0
6 <NA>  <NA>  <NA>  <NA>  <NA>   lh_pallidum <NA>  pallidum                 13.4
7 <NA>  <NA>  <NA>  <NA>  <NA>   lh_putamen  <NA>  putamen                  18.4
8 <NA>  <NA>  <NA>  <NA>  <NA>   lh_thalamu… <NA>  thalamuspr…              17.4
# ℹ 2 more v

In [84]:
%%R -i regional_vascular_data

### Pearson FC in brain ###
regional_vascular_data %>%
  ungroup() %>%
  mutate(label = glue("lh_{Base_Region}")) %>%
  left_join(., as_tibble(dk)) %>%
  ggseg(atlas = dk, mapping = aes(fill = Venous_Vascular),
        position = "stacked", colour = "black", hemisphere="left",
        linewidth=0.5) +
  theme_void() +
  labs(fill = "Mean venous density") +
  theme(plot.title = element_blank(),
        legend.position = "bottom") +
  scale_fill_gradient(low = "white", high = "red", na.value="white")


ggsave("venous.png", width=5, height=3, units='in', dpi=300)

Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
In addition: Warning message:
Some data not merged properly. Check for naming errors in data:
  atlas type  hemi  side  region label       roi   Base_Region Arterial_Vascular
  <chr> <chr> <chr> <chr> <chr>  <glue>      <chr> <chr>                   <dbl>
1 <NA>  <NA>  <NA>  <NA>  <NA>   lh_accumbe… <NA>  accumbensa…              41.7
2 <NA>  <NA>  <NA>  <NA>  <NA>   lh_amygdala <NA>  amygdala                 22.4
3 <NA>  <NA>  <NA>  <NA>  <NA>   lh_brainst… <NA>  brainstem                46.9
4 <NA>  <NA>  <NA>  <NA>  <NA>   lh_caudate  <NA>  caudate                  14.9
5 <NA>  <NA>  <NA>  <NA>  <NA>   lh_hippoca… <NA>  hippocampus              21.0
6 <NA>  <NA>  <NA>  <NA>  <NA>   lh_pallidum <NA>  pallidum                 13.4
7 <NA>  <NA>  <NA>  <NA>  <NA>   lh_putamen  <NA>  putamen                  18.4
8 <NA>  <NA>  <NA>  <NA>  <NA>   lh_thalamu… <NA>  thalamuspr…              17.4
# ℹ 2 more v

## Is homotopic connectivity correlated with vascular innervation?

In [68]:
HoFC_and_vascular_info = pd.merge(pearson_homotopic_res_mean, regional_vascular_data, on='Base_Region', how='left')

HoFC_and_vascular_info.head()

,Base_Region,Mean_Pearson_R,Arterial_Vascular,Venous_Vascular
0,bankssts,0.314670,5.629985,39.885609
1,caudalanteriorcingulate,0.213912,20.057773,38.279240
2,caudalmiddlefrontal,0.330858,17.089778,32.134382
3,cuneus,0.644840,15.818715,69.034659
4,entorhinal,0.029524,27.140752,80.060211


In [86]:
%%R -i HoFC_and_vascular_info -w 800 -h 400

HoFC_and_vascular_info %>% 
    pivot_longer(cols=c("Arterial_Vascular", "Venous_Vascular"), 
                 names_to="Vascular_Type", values_to="Mean_Vascular_Density") %>%
    ggplot(aes(x=Mean_Vascular_Density, y=Mean_Pearson_R)) +
    geom_point(color='red', size=2) +
    facet_wrap(Vascular_Type ~ ., scales='free') +
    geom_smooth(method='lm', se=FALSE, color="black") +
    xlab("Mean Vascular Density") +
    ylab("Mean Pearson Homotopic FC") +
    ggpubr::stat_cor(method = "spearman", cor.coef.name='rho', label.y = 0.1, size=5) +
    theme(legend.position='none',
          strip.background = element_blank(),
          strip.text = element_text(size=14, face='bold'))


ggsave("scatters.png", width=7, height=3.5, units='in', dpi=300)

`geom_smooth()` using formula = 'y ~ x'
